In [1]:
# Journey to the Centre of the Earth
! wget https://www.gutenberg.org/cache/epub/18857/pg18857.txt

--2023-09-07 20:50:50--  https://www.gutenberg.org/cache/epub/18857/pg18857.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 518044 (506K) [text/plain]
Saving to: ‘pg18857.txt’

pg18857.txt         100%[===================>] 505.90K  1.44MB/s    in 0.3s    

2023-09-07 20:50:55 (1.44 MB/s) - ‘pg18857.txt’ saved [518044/518044]



In [2]:
! pip install tokenizers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00


In [3]:

from tokenizers import models , Tokenizer, decoders , pre_tokenizers , trainers , normalizers


In [4]:
# we will make the tokenizer object from the BPE tokenizer class

tokenizer = Tokenizer(models.BPE())


# For the normalization part we will add the Lowercase function
tokenizer.normalizer = normalizers.Sequence([normalizers.Lowercase()])

# the pre_tokenizer attribute is set to be as ByteLevel to ensure we have bytes as our input
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# The decoder attribute must be also set to ByteLevelDecoder to be able to decode correctly
tokenizer.decoder = decoders.ByteLevel()


In [5]:
trainer = trainers.BpeTrainer(vocab_size=50000,
                     inital_alphabet=pre_tokenizers.ByteLevel.alphabet(),
                     special_tokens=[
                                      "<s>",
                                      "<pad>",
                                      "</s>",
                                      "<unk>",
                                      "<mask>"])
tokenizer.train(["/content/pg18857.txt"], trainer)

In [6]:
! mkdir our_GPT_tokenizer

In [7]:
tokenizer.save("our_GPT_tokenizer/tokenizer.json")

In [8]:
from transformers import GPT2TokenizerFast, GPT2Config, TFGPT2LMHeadModel

In [9]:
our_tokenizer = GPT2TokenizerFast.from_pretrained("our_GPT_tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
our_tokenizer.add_special_tokens({
        "eos_token": "</s>",
        "bos_token": "<s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>",
        "mask_token": "<mask>"
})

0

In [11]:
our_tokenizer.eos_token_id

2

In [12]:
en = our_tokenizer("<s> hello AI Mastery </s>")
en

{'input_ids': [0, 148, 118, 51, 83, 45, 91, 111, 1696, 72, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokens = our_tokenizer.convert_ids_to_tokens(en.input_ids)
print(tokens)

['<s>', 'Ġhe', 'll', 'o', 'Ġa', 'i', 'Ġm', 'as', 'tery', 'Ġ', '</s>']


In [14]:
print(our_tokenizer.backend_tokenizer.normalizer)

In [15]:
# Let’s now take a look at the pretokenization:
print(our_tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("<s> hi there </s>"))

[('<', (0, 1)), ('s', (1, 2)), ('>', (2, 3)), ('Ġhi', (3, 6)), ('Ġthere', (6, 12)), ('Ġ</', (12, 15)), ('s', (15, 16)), ('>', (16, 17))]


In [16]:
config = GPT2Config(
  vocab_size=our_tokenizer.vocab_size,
  bos_token_id=our_tokenizer.bos_token_id,
  eos_token_id=our_tokenizer.eos_token_id)

model = TFGPT2LMHeadModel(config)

In [17]:
print(config)

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.33.1",
  "use_cache": true,
  "vocab_size": 13343
}



In [18]:
with open("/content/pg18857.txt", "r", encoding='utf-8') as f:
    content = f.readlines()

In [41]:
content[156]

'truly so wonderful that even now I am bewildered when I think of them.\n'

In [56]:
content_p = []
for c in content:
    if len(c)>10:
        content_p.append(c.strip() )

content_p = " ".join(content_p) + our_tokenizer.eos_token



In [57]:
content_p[1000:1300]

' of Jules Verne. First published in England by Griffith and Farran, 1871, this edition is not a translation at all but a complete re-write of the novel, with portions added and omitted, and names changed. The most reprinted version, it is entered into Project Gutenberg for reference purposes only. A'

In [58]:
tokenized_content = our_tokenizer.encode(content_p)

In [59]:
len(tokenized_content)

104664

In [60]:
sample_len = 100
examples = []
for i in range(0, len(tokenized_content)):
    examples.append(tokenized_content[i:i + sample_len])

train_data = []
labels = []
for example in examples:
    train_data.append(example[:-1])  # 0 -->  99
    labels.append(example[1:])    #    1 --> 100

In [61]:
len(train_data)

104664

In [62]:
import tensorflow as tf

buffer = 100
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices((train_data[:100000],labels[:100000]))
dataset = dataset.shuffle(buffer).batch(batch_size, drop_remainder=True)

In [63]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(16, 99), dtype=tf.int32, name=None), TensorSpec(shape=(16, 99), dtype=tf.int32, name=None))>

In [64]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [65]:
epochs = 5
model.fit(dataset, epochs=epochs)

Epoch 1/5
6250/6250 [==============================] - 2576s 406ms/step - loss: 5.1409 - accuracy: 0.1845
Epoch 2/5
6250/6250 [==============================] - 2542s 407ms/step - loss: 4.4122 - accuracy: 0.2246
Epoch 3/5
6250/6250 [==============================] - 2542s 407ms/step - loss: 3.7538 - accuracy: 0.2836
Epoch 4/5
6250/6250 [==============================] - 2543s 407ms/step - loss: 2.8431 - accuracy: 0.4025
Epoch 5/5
6250/6250 [==============================] - 2541s 407ms/step - loss: 1.6049 - accuracy: 0.6362


In [74]:
model.save_pretrained("Mini_GPT2_model")

In [71]:
from transformers import WEIGHTS_NAME, CONFIG_NAME, TF2_WEIGHTS_NAME
our_tokenizer.save_pretrained("Mini_GPT2_tok")

('Mini_GPT2_tok/tokenizer_config.json',
 'Mini_GPT2_tok/special_tokens_map.json',
 'Mini_GPT2_tok/vocab.json',
 'Mini_GPT2_tok/merges.txt',
 'Mini_GPT2_tok/added_tokens.json',
 'Mini_GPT2_tok/tokenizer.json')

In [75]:
from transformers import GPT2TokenizerFast, TFGPT2LMHeadModel

tokenizer = GPT2TokenizerFast.from_pretrained("Mini_GPT2_tok")
model_reloaded = TFGPT2LMHeadModel.from_pretrained("Mini_GPT2_model")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at Mini_GPT2_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [76]:
def generate(prompt, model):
    input_token_ids = tokenizer.encode(prompt, return_tensors='tf')
    output = model.generate(
        input_token_ids,
        pad_token_id = tokenizer.pad_token_id,
        eos_token_id = tokenizer.eos_token_id,
        max_length = 128,
        num_beams = 5,
        temperature = 0.7,
        no_repeat_ngram_size=2,
        num_return_sequences=1
    )
    return tokenizer.decode(output[0])

In [77]:
generate("it is an exciting idea " , model_reloaded)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'it is an exciting idea  chorus. we have reached the superstitious character of san vicenza, where hans demanded the price of his thirteenth week of service. my uncle paid him, with very many warm shakes of the hand. at that moment, if he did not indeed quite share our natural emotion, he allowed his feelings so far to give way as to indulge in an extraordinary expression for him. with the tips of two fingers he gently pressed our hands and smiled. chapter 44 the journey ended this is the final conclusion of a narrative which will be probably disbelieved even by people who are astonished at nothing. i am, however, i do not'